In [1]:
from experta import *

In [2]:
class Age(Fact):
    pass

class Height(Fact):
    pass

class Weight(Fact):
    pass

class Gender(Fact):
    pass 

class Calories(Fact):
    pass

class Competing(Fact):
    pass

class Intensity(Fact):
    pass

class DietPlan(Fact):
    pass

class BMI (Fact):
    pass

class setOptions(Fact):
    pass

class Option(Fact):
    foodSet = Field(str, default="")
    option = Field(str, default="")
    calories = Field(str, default="")

class Set(Fact):
    setCategory = Field(str, default="")
    setFoods = Field(list, default=[])
    setCalories = Field(str, default="")
    

class Ask(Fact):
    pass

In [3]:
class SystemDiet(KnowledgeEngine):
    @DefFacts()
    def system_rules(self):
        yield Option(foodSet="casual",option="C",calories=">=2500")
        yield Option(foodSet="casual",option="B",calories=">=1500 & <2500")
        yield Option(foodSet="casual",option="A",calories="<1500") 
        
        yield Option(foodSet="light",option="F",calories=">=3500 & <4500")
        yield Option(foodSet="light",option="E",calories=">=2500 & <3500")
        yield Option(foodSet="light",option="D",calories="<2500")
        
        yield Option(foodSet="moderate",option="I",calories=">=3500")
        yield Option(foodSet="moderate",option="H",calories=">=2500 & <3500")
        yield Option(foodSet="moderate",option="G",calories="<2500")
        
        yield Option(foodSet="intensive",option="L",calories=">=4500")
        yield Option(foodSet="intensive",option="K",calories=">=3500 & <4500")
        yield Option(foodSet="intensive",option="J",calories="<3500")
        
        
        yield Set(setCategory="A",setFoods=["Fatty/Oily Rice","charcoal-grilled chicken","chicken rice"],
                  setCalories="<1500")
        yield Set(setCategory="B",setFoods=["Flattened bread","arab rice mandy","Mamak-style noodles,"],
                  setCalories=">=1500 & <2500")
        yield Set(setCategory="C",setFoods=["rice with salad + Flattened bread","rice on the shoulder","Chicken Chop"],
                  setCalories=">=2500")
        
        yield Set(setCategory="D",setFoods=["Fatty/Oily Rice","charcoal-grilled chicken","satay, fried wet"],
                  setCalories="<2500")
        yield Set(setCategory="E",setFoods=["Flattened bread","rice dish flavored with spices and rose water","Spaghetti"],
                  setCalories=">=2500 & <3500")
        yield Set(setCategory="F",setFoods=["Flat rice noodles. + Flattened bread","rice on the shoulder","Fried rice"],
                  setCalories=">=3500 & <4500")
        
        yield Set(setCategory="G",setFoods=["Fatty/Oily Rice + Flattened bread","charcoal-grilled chicken","Pizza"],
                  setCalories="<2500")
        yield Set(setCategory="H",setFoods=["rice with salad + Flattened bread","Chicken Fried Rice + chewy and slippery","Fried rice with egg"],
                  setCalories=">=2500 & <3500")
        yield Set(setCategory="I",setFoods=["Flat rice noodles. + Fatty/Oily Rice + Flattened bread","arab rice mandy + Vermicelli soup",
                  "Spaghetti + Chicken Chop + satay, fried wet"],setCalories=">=3500")
        
        yield Set(setCategory="J",setFoods=["Flat rice noodles. + Fatty/Oily Rice + Cereal",
                  "rice on the shoulder + Nasi Ayam Goreng + Bihun Sup","satay, fried wet"],setCalories="<3500")
        yield Set(setCategory="K",setFoods=["Flattened bread + rice with salad + Cereal",
                  "rice dish flavored with spices and rose water + charcoal-grilled chicken + chewy and slippery","Mamak-style noodles, + Fried rice with egg + Pizza"],
                  setCalories=">=3500 & <4500")
        yield Set(setCategory="L",setFoods=["Flattened bread + Flattened bread + Flat rice noodles. + Cereal",
                  "rice on the shoulder + Chicken Fried Rice + Bihun Sup","Nasi Goreng Pattaya + Mamak-style noodles, + Chicken Chop"],
                  setCalories=">=4500")

    @Rule()
    def start(self):
        print("The system has been started")

    @Rule(AS.f1<<Gender(MATCH.g),
      AS.f2<<Age(MATCH.a),
      AS.f3<<Height(MATCH.h),
      AS.f4<<Weight(MATCH.w))
    def gender(self,g,a,h,w):
        if(g=="male"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))+5)
            self.declare(Calories(value = caloriesFloat))
            print("You are a " + g)

        elif(g =="female"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))+161)
            self.declare(Calories(value = caloriesFloat))
            print("You are a " + g)
        else:
            print("Invalid gender")
        self.declare(Ask("competition-participation"))

    @Rule(AS.f1<<Ask("competition-participation"))
    def if_competing(self,f1):
        self.retract(f1)
        if(Competing(MATCH.c) == 'y'):
           print("You are competing!")
        elif(Competing(MATCH.c) == 'n'):
           print("You are not competing!")


    @Rule(AS.f1 << Competing('n'))
    def not_cometing(self,f1):
        self.declare(DietPlan("casual"))

    @Rule(AS.f1<<Competing("y"),
      AS.f2<<Height(MATCH.h),
      AS.f3<<Height(MATCH.w))
    def if_competing(self,h,w):
        heightInM = int(h)/100
        bmi = int(w)/pow(heightInM,2)
        if(bmi < 18.5):
            self.declare(BMI("underweight"))
            print("You are underweight")
        elif(bmi >= 18.5 and bmi < 25.0):
            self.declare(BMI("normal"))
            print("You are normal")
        elif(bmi >= 25.0):
            self.declare(BMI("overweight")) 
            print("You are fat lol")


    @Rule(AS.f1<<Intensity("low"),OR(BMI("normal"),
              BMI("overweight")
             ))
    def low_normal_overweight(self):
        self.declare(DietPlan("light"))

    @Rule(AS.f1<<Intensity("low"),BMI("underweight")
      )
    def low_underweight(self):
         self.declare(DietPlan("moderate"))
    @Rule(AS.f1 << Intensity("moderate"))
    def moderate_normal(self):
        self.declare(DietPlan("moderate"))

    @Rule(AS.f1 << Intensity("high"),
          BMI("overweight"))
    def high_overweight(self):
        self.declare(DietPlan("moderate"))


    @Rule(AS.f1 << Intensity("high"),
          OR(
              BMI("normal"),
              BMI("underweight")
             )
          )
    def high_normal_underweight(self):
        self.declare(DietPlan("intensive"))

    ''' Rules Section - Display Diet Plans '''
    
    
    @Rule(DietPlan(MATCH.f),
          salience=2)
    def display_assistance(self, f):
        print("For a " + f + " diet plan, these are the options")
    
    @Rule(AS.f1 << DietPlan(MATCH.f),
          Option(foodSet = MATCH.f,
                 option = MATCH.o,
                 calories = MATCH.c),salience=1)
    def display_moderate_plans(self, o, c):
        print(o + "(" + c + "kcal)")
    
    @Rule(AS.f1 << DietPlan("casual"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "casual",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_casual(self, cc):
        if (cc < 1500.0):
            self.declare(setOptions("A"))
        elif (cc >= 1500.0 and cc < 2500.0):
            self.declare(setOptions("B"))
        elif (cc >=2500.0):
            self.declare(setOptions("C"))
            
    @Rule(AS.f1 << DietPlan("light"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "light",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_light(self, cc):
        if (cc < 1500.0):
            self.declare(setOptions("D"))
        elif (cc >= 1500.0 and cc < 2500.0):
            self.declare(setOptions("E"))
        elif (cc >= 2500.0):
            self.declare(setOptions("F"))
    
    @Rule(AS.f1 << DietPlan("moderate"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "moderate",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_moderate(self, cc):
        if (cc < 2500.0):
            self.declare(setOptions("G"))
        elif (cc >= 2500.0 and cc < 3500.0):
            self.declare(setOptions("H"))
        elif (cc >= 3500.0):
            self.declare(setOptions("I"))
            
    @Rule(AS.f1 << DietPlan("intensive"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "intensive",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_intensive(self, cc):
        if (cc < 3500.0):
            self.declare(setOptions("G"))
        elif (cc >= 3500.0 and cc < 4500.0):
            self.declare(setOptions("H"))
        elif (cc >= 4500.0):
            self.declare(setOptions("I"))

    @Rule(setOptions(MATCH.o),
          Calories(value=MATCH.cc),
          Set(setCategory = MATCH.o,
              setFoods = MATCH.f,
              setCalories = MATCH.c))
    def display_foods(self, o, cc, f, c):
        print("Considering you need " , cc , "kcal, you are recommended eat these in set " + o +": ")
        for z in f:
            print(z + "\t", end="")
        print()
        print("All foods in this set contains calories in the " + c + " range.")

In [4]:
from fastapi import FastAPI
import asyncio 
from uvicorn import Config , Server
from pydantic import BaseModel
app = FastAPI()

class UserInput(BaseModel):
    age : str
    height : str
    weight : str
    gender : str
    competing : str
    intensity : str

@app.get('/')
async def index():
    return {"Message":"Hello World"}

config  = Config(app)
server  =  Server(config=config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

@app.post('/age')
async def age(user_input: UserInput):
    age = user_input.age
    height = user_input.height
    weight = user_input.weight
    gender = user_input.gender
    competing = user_input.competing
    intensity = user_input.intensity
    engine = SystemDiet()
    engine.reset()
    engine.declare(Age(age), Height(height), Weight(weight), Gender(gender), Competing(competing), Intensity(intensity))
    resultRun = engine.run()
    print("this is resultRun")
    print(resultRun)
    result = engine.facts
    return {"result":result,"resultRun":resultRun}
 

INFO:     Started server process [5336]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


You are fat lol
You are a male
The system has been started
this is resultRun
None
INFO:     127.0.0.1:59382 - "POST /age HTTP/1.1" 200 OK
